## **Demo Notebook**

This quick notebook will show how to

* load config
* create the dataset
* make a model
* load pretrained weights

This notebook should be run directly from the `scripts/` directory.

### **Config**

Make sure to change the following paths
* `DATASET_DIR`
* `LABELS_DIR`

In [1]:
from hydra import core, initialize, compose
from omegaconf import OmegaConf
import torch

device = torch.device('cpu')

core.global_hydra.GlobalHydra.instance().clear()        # required for Hydra in notebooks

initialize(config_path='../config')

# Add additional command line overrides
cfg = compose(
    config_name='config', 
    overrides=[
        'data.json_dir=../data/'
    ]
)

# resolve config references
# OmegaConf.resolve(cfg)

print(list(cfg.keys()))

['experiment', 'loader', 'optimizer', 'scheduler', 'trainer', 'model', 'data', 'loss', 'metrics']


/tmp/ipykernel_3236370/171547621.py:9: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path='../config')


In [2]:
cfg['data']['json_dir']

'../data/'

In [3]:
torch.cuda.is_available()

True

### **Dataset Setup**

In [12]:
import torch
import numpy as np

from cross_view_transformer.common import setup_carla_experiment, load_backbone


model_module, data_module = setup_carla_experiment(cfg)

SUBSAMPLE = 10

dataset = data_module.val_dataset
# dataset = torch.utils.data.ConcatDataset(dataset)
dataset = torch.utils.data.Subset(dataset, range(0, len(dataset), SUBSAMPLE))

loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False, num_workers=2)

len(dataset)


Loaded pretrained weights for efficientnet-b4


121

In [13]:
current = next(iter(loader))

In [14]:
current['bev'].shape  

torch.Size([4, 3, 500, 500])

### **Network Setup**

In [15]:
from pathlib import Path

# Download a pretrained model (13 Mb)
CHECKPOINT_PATH = '../logs/cross_view_transformers_road/0706_193619/checkpoints/model-v1.ckpt'

if Path(CHECKPOINT_PATH).exists():
    network = load_backbone(CHECKPOINT_PATH)
    print("found presaved")
else:
    network = model_module.backbone

    print(f'{CHECKPOINT_PATH} not found. Using randomly initialized weights.')

Loaded pretrained weights for efficientnet-b4
found presaved


### **Run the Model**

This cell will save the model predictions and render them as a gif.

Note: colors will be ugly since gif only supports 256 colors total

In [16]:
# load IOU metric
from cross_view_transformer.metrics import IoUMetric

iou_metric = IoUMetric(label_indices=cfg.data.label_indices).to(device)


In [17]:
cfg.data.label_indices

[[1, 2]]

In [18]:
for batch in loader:
    batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}
    # Forward pass and compute predicted outputs
    # try:
    #     with torch.no_grad():
    #         predicted_outputs = network(batch)['bev'].to(device)
    # except RuntimeError:
    #     devices = {k: v.device if torch.is_tensor(v) else 'not_tensor' for k, v in batch.items()}
    with torch.no_grad():
        predicted_outputs = network(batch)['bev'].to(device)

    # Update the IoU metric
    iou_metric.update(predicted_outputs, batch)

/home/dxa230028/.local/lib/python3.11/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [21]:
iou_values = iou_metric.compute()

iou_040 = iou_values['@0.40']
iou_050 = iou_values['@0.50']

In [22]:
print("Val IoUs for road segmentation")
print(f"For Threshold 0.40: {iou_040}")
print(f"For Threshold 0.50: {iou_050}")

Val IoUs for road segmentation
For Threshold 0.40: 0.9582176804542542
For Threshold 0.50: 0.9617186188697815
